<a href="https://colab.research.google.com/github/yeonghun00/stock_public/blob/main/stock_support.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime

In [2]:
# 관련 업종
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}

def get_related(code:str='005930'):
  url = 'https://finance.naver.com/item/main.nhn?code=' + code
  result = requests.get(url, headers = headers)
  bs_obj = BeautifulSoup(result.content, "html.parser")
  th = bs_obj.find_all("th", {'scope':"col"})
  th = [x.get_text() for x in th][-12:-7]
  elements = [x[-6:] for x in th]
  return elements

get_related()

['005930', '000660', '357780', '058470', '240810']

In [3]:
# 업종
dic = {'에너지': 'WI100', '화학': 'WI110', '비철금속': 'WI200', '철강': 'WI210', '건설': 'WI220', '기계': 'WI230', '조선': 'WI240', '상가,자본재': 'WI250', \
       '운송': 'WI260', '자동차': 'WI300', '화장품,의류': 'WI310', '호텔,레저': 'WI320', '미디어,교육': 'WI330', '소매(유통)': 'WI340', '필수소비재': 'WI400', \
       '건강관리': 'WI410', '은행': 'WI500', '증권': 'WI510', '보험': 'WI520', '소프트웨어': 'WI600', 'IT하드웨어': 'WI610', '반도체': 'WI620', 'IT가전': 'WI630', \
       '디스플레이': 'WI640', '전기통신서비스': 'WI700', '유틸리티': 'WI800'}

def get_sector(code):
  url = 'http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt=' + \
  str(int(''.join(str(datetime.date.today()).split('-'))) - 10) + '&sec_cd=' + dic[code]
  result = requests.get(url, headers = headers)
  json_list = result.json() 

  df = pd.DataFrame(columns=['code', 'name', 'sector_name', 'idx_name'])

  for json in json_list['list']:
    code = json['CMP_CD']
    name = json['CMP_KOR']
    sector_name = json['SEC_NM_KOR']
    idx_name = json['IDX_NM_KOR']
    df = df.append({'code':code, 'name':name, 'sector_name':sector_name, 'idx_name':idx_name}, ignore_index=True)

  return df

In [121]:
# 검색 상위

def get_searched():
  url = 'https://finance.naver.com/sise/lastsearch2.nhn'
  result = requests.get(url, headers = headers)
  bs_obj = BeautifulSoup(result.content, "html.parser")
  a = bs_obj.find_all('a',{'class', 'tltle'}) # title nono tltle
  return [x.get_text() for x in a]

In [ ]:
def get_highest():
  

In [42]:
def get_higher(market='ks', num=500):
  i = 0
  if market == 'kq':
    i = 1
  url = 'https://finance.naver.com/sise/sise_rise.nhn?sosok=' + str(i)
  result = requests.get(url, headers = headers)
  bs_obj = BeautifulSoup(result.content, "html.parser")
  a = bs_obj.find_all('a',{'class', 'tltle'}) # title nono tltle
  code = [x['href'][-6:] for x in a]
  name = [x.get_text() for x in a]
  
  d = {'code':code, 'name':name}
  df = pd.DataFrame(d)

  return df.loc[:num]

In [43]:
get_higher('ks')

,code,name
0,009275,신원우
1,35320K,대덕전자1우
2,383220,F&F
3,009270,신원
4,024900,덕양산업
...,...,...
467,272580,TIGER 단기채권액티브
468,278620,ARIRANG 단기채권액티브
469,306540,HANARO 단기통안채
470,363510,SMART KIS단기통안채


In [52]:
url = 'https://finance.naver.com/sise/sise_rise.nhn?sosok=0'
result = requests.get(url, headers = headers)
bs_obj = BeautifulSoup(result.content, "html.parser")
a = bs_obj.find_all('a',{'class', 'tltle'}) # title nono tltle
code = [x['href'][-6:] for x in a]
name = [x.get_text() for x in a]
change = [x.replace('\n','').replace('\t','') for x in [span.get_text() for span in spans] if "%" in x ]
d = {'code':code, 'name':name, 'change':change}

In [116]:
url = 'https://finance.naver.com/sise/'
result = requests.get(url, headers = headers)
bs_obj = BeautifulSoup(result.content, "html.parser")
a = bs_obj.find_all('a',{'class', 'tltle'}) # title nono tltle
t = [(x['onclick'][7:]).split(',')[3] for x in a]
t = list(map(lambda x: [s for s in x if s.isdigit()], t))
idxes = list(map(lambda x: int(''.join(x)), t))
codes = [x.get_text() for x in a]
li = list(zip(idxes,codes))

['삼성전자',
 '카카오',
 'NAVER',
 'HMM',
 '한전기술',
 'LG화학',
 '코오롱인더우',
 'KODEX 건설',
 'SK하이닉스',
 '케이씨에스',
 '셀트리온',
 '두산중공업',
 'PN풍년',
 '한국전력',
 'SK바이오사이언스',
 '삼성바이오로직스',
 'ARIRANG 고배당주',
 '시너지이노베이션',
 '한국조선해양',
 '현대차',
 'SK아이이테크놀로지',
 '하이브',
 '에스티팜',
 'KODEX 배당성장',
 '웹젠',
 '엘앤에프',
 '신풍제약',
 '금호석유',
 '진원생명과학',
 '삼성전기']